In [82]:
from ctse.sysdata.csv_ctse_strategy_backtest_data import csvCTSEStrategyBacktestData
from sysdata.csv.csv_multiple_prices import csvFuturesMultiplePricesData
import pandas as pd
import numpy as np
from ctse.sysobjects.ctse_strategy_backtest import ctseStrategyBacktest
from sysdata.csv.csv_instrument_data import csvFuturesInstrumentData
from itables import show
def select(se, start, end):
    return se[(se.index >= start) &(se.index <= end)]
ins = pd.Series(csvFuturesInstrumentData('ctse.data.csvconfig').get_list_of_instruments()).sample(1).iloc[0]
ins = 'SC'
print(ins)

notional_capital = 2000_0000
'''
@ discrete system
@ open price time series
    | current period |--next price--| next period |

Time Index Symantics:
    Forecast: ex-ante
    Position: ex-post
    Transaction: ex-post, Position change
    PnL: ex-post
'''

meta_data = csvFuturesInstrumentData('ctse.data.csvconfig').get_instrument_data(ins).meta_data
pointsize = meta_data.Pointsize
transaction_cost_percentage = meta_data.Percentage
print(pointsize)
print(transaction_cost_percentage)

backtest = csvCTSEStrategyBacktestData().read('1_B', ins, 'dma_long', cols=[ctseStrategyBacktest.STATE_FLAG, ctseStrategyBacktest.OPEN, ctseStrategyBacktest.CLOSE, ctseStrategyBacktest.OPERATE, ctseStrategyBacktest.NAV])
backtest.loc[:, 'forecast'] = backtest[ctseStrategyBacktest.STATE_FLAG].replace(np.nan, 0)
backtest.loc[:, 'entry_transaction'] = ((backtest.forecast.shift(1).diff() == 1) * 1)
backtest.loc[:, 'holding'] = backtest.forecast.shift(1)
backtest.loc[:, 'exit_transaction'] = ((backtest.forecast.shift(1).diff() == -1) * 1)
fill = backtest.loc[backtest[ctseStrategyBacktest.OPERATE].replace(np.nan, '').str.contains('止损')]
backtest.loc[fill.index, 'tr_fill_price'] = fill[ctseStrategyBacktest.OPERATE].apply(lambda s: float(s.split('_')[-1]))
backtest = backtest.drop(ctseStrategyBacktest.STATE_FLAG, axis=1)

contract = csvFuturesMultiplePricesData('ctse.data.multiple_prices_csv')._get_multiple_prices_without_checking(ins, check_data=False)[['PRICE_CONTRACT', 'PRICE', 'CLOSE', 'FORWARD_CLOSE']]
# rolling
# df.loc[df['FORWARD'].notna(), 'FORWARD'] = df['CLOSE']
# df.loc[df['FORWARD'].shift(-1).notna(), 'FORWARD'] = df.PRICE.shift(-1)
# df.loc[:, 'one_contract_rolling_cost'] = df['FORWARD_CLOSE'].diff().shift(-1) * df.CLOSE / df.FORWARD_CLOSE

df = pd.concat([backtest, contract], axis=1)

df.loc[:, 'fixed_capital_continious_position'] = 0
df.loc[df.holding == 1, 'fixed_capital_continious_position'] = np.nan
df.loc[df.entry_transaction == 1, 'fixed_capital_continious_position'] = (df.entry_transaction * notional_capital / (df.PRICE * pointsize))
df.fixed_capital_continious_position.ffill(inplace=True)

# PnL
##
df.loc[:, 'entry_transaction_pnl'] = (df.entry_transaction * df.fixed_capital_continious_position * df.PRICE * pointsize * -transaction_cost_percentage ).shift(-1)

## rolling
# df.loc[:, 'one_contract_rolling_cost'] = (-df.FORWARD_CLOSE + df.CLOSE) * pointsize
# df.loc[:, 'rolling_pnl'] = df.one_contract_rolling_cost * df.fixed_capital_continious_position




## Fill for stop loss order
df.loc[:, 'Fill_Price'] = df.tr_fill_price / df[ctseStrategyBacktest.OPEN] * df['PRICE']
df.loc[:, 'one_contract_pnl_at_fill'] = (df['Fill_Price'] - df.PRICE) * pointsize
df.loc[:, 'stop_loss_pnl'] = df.one_contract_pnl_at_fill * df.fixed_capital_continious_position

## 
df.loc[:, 'exit_transaction_pnl'] = (df.exit_transaction * df.fixed_capital_continious_position.shift(1) * df.one_contract_transaction_cost).shift(-1)






# same contract
df.loc[:, 'same_contract_pnl'] = df.PRICE.shift(-1).diff()

#
df.loc[:, 'contract_pnl_adjusted'] = df.loc[:, 'same_contract_pnl']
df.loc[df['contract_pnl_at_fill'].notna(), 'contract_pnl_adjusted'] = df['contract_pnl_at_fill']
df.loc[df.one_contract_rolling_cost.notna(), 'contract_pnl_adjusted'] = df.one_contract_rolling_cost
df.loc[:, 'contract_pnl_adjusted'] = df.contract_pnl_adjusted * pointsize

# df.loc[:, 'discrete_position'] = (df.forecast.shift(1) * notional_capital // (df.PRICE * pointsize))
df.loc[:, 'fixed_capital_gross_pnl'] = (df.fixed_capital_continious_position * df.contract_pnl_adjusted)
df.loc[:, 'fixed_capital_net_pnl'] = df.fixed_capital_gross_pnl #+ df.fixed_capital_transaction_pnl

# df = df.loc[:, ctseStrategyBacktest.NAV] = column_nav


t1 = (df.fixed_capital_net_pnl / notional_capital)
# t1 = select(t1, '2020-12-01', '2021-02-01')0
t2 = df.tr_nav_of_product.pct_change()
# t2 = select(t2, '2020-12-01', '2021-02-01')
# t1.plot(legend=True)
# t2.plot()

%matplotlib widget

start = '2019-02-01'
end = '2019-03-01'

show(select(pd.concat([t1, t2], axis=1), start, end))

# pretty
column_nav = df.pop(ctseStrategyBacktest.NAV)
df.loc[:, ctseStrategyBacktest.NAV] = column_nav
show(select(df, start, end))


(1+t1).cumprod().plot(legend=True)
(1+t2).cumprod().plot(legend=True)


SC
1000
0.0002


,tr_the_open_price,tr_the_close_price,tr_operate,tr_nav_of_product,forecast,entry_transaction,holding,exit_transaction,tr_fill_price,PRICE_CONTRACT,PRICE,CLOSE,FORWARD_CLOSE,one_contract_rolling_cost,fixed_capital_continious_position,entry_transaction_pnl,Fill_Price,one_contract_pnl_at_fill,stop_loss_pnl
index,,,,,,,,,,,,,,,,,,,
2018-04-13,425.000000,427.000000,NaN,1.000000,0.0,0.0,NaN,0.0,NaN,20180900,425.0,427.0,NaN,NaN,0.000000,-0.0,NaN,NaN,NaN
2018-04-16,427.800000,424.400000,NaN,1.000000,0.0,0.0,0.0,0.0,NaN,20180900,427.8,424.4,NaN,NaN,0.000000,-0.0,NaN,NaN,NaN
2018-04-17,426.300000,427.400000,NaN,1.000000,0.0,0.0,0.0,0.0,NaN,20180900,426.3,427.4,NaN,NaN,0.000000,-0.0,NaN,NaN,NaN
2018-04-18,422.700000,429.700000,NaN,1.000000,0.0,0.0,0.0,0.0,NaN,20180900,422.7,429.7,NaN,NaN,0.000000,-0.0,NaN,NaN,NaN
2018-04-19,435.100000,440.600000,NaN,1.000000,0.0,0.0,0.0,0.0,NaN,20180900,435.1,440.6,NaN,NaN,0.000000,-0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-25,489.975557,497.452023,0_485.04948460037235,1.116882,1.0,1.0,1.0,0.0,NaN,20221000,734.0,745.2,NaN,NaN,27.247956,-0.0,NaN,NaN,NaN
2022-08-26,496.984744,493.980807,平仓(止损)_492.26375262625584,1.106051,0.0,0.0,1.0,0.0,492.263753,20221000,744.5,740.0,NaN,NaN,27.247956,-0.0,737.427795,-7072.204988,-192703.133188
2022-08-29,488.974244,486.971619,0_0.0,1.106051,0.0,0.0,0.0,1.0,NaN,20221000,732.5,729.5,NaN,NaN,0.000000,-0.0,NaN,NaN,NaN


In [77]:
df = pd.DataFrame({
    ctseStrategyBacktest.NAV:[1,2],
    'b':[1,2],
})
df.pop(ctseStrategyBacktest.NAV)

0    1
1    2
Name: tr_nav_of_product, dtype: int64